## Importando bibliotecas

In [1]:
import pyomo.environ as pyo # Fornece modelos de programação linear
import pandas as pd         # Tratamento de dados, leitura dos arquivos de entrada
import numpy as np          # Facilitar operações com números
import urllib.request       # Obter a solicitação da url para a obtenção das distâncias via API
import json                 # Habilita a leitura do arquivo json obtido com a API

## Leitura do arquivo de entrada

In [2]:
df = pd.read_excel("OrcPedPequeno.ods")

In [3]:
df.head()

,OrcPed,Emissão,Endereço,Bairro,Cidade,Estado,Peso
0,1,2001-01-01 00:00:00,"RUA CORONEL DUARTE DA SILVEIRA, 605",CASTRIOTO,PETRÓPOLIS,RJ,0.0
1,2,01/20/06 00:00:00,RUA BENJAMIN CONSTANT 213,CENTRO,PETRÓPOLIS,RJ,2.0
2,3,09/01/00 00:00:00,"RUA PAULINO AFONSO , 477",CENTRO,PETRÓPOLIS,RJ,1.2
3,4,09/13/00 00:00:00,"RUA 24 DE MAIO, 517",CENTRO,PETRÓPOLIS,RJ,3.0
4,5,09/19/00 00:00:00,RUA ALICE HERVE 356,CAPELA,PETRÓPOLIS,RJ,0.5


## Obter as distâncias entre cada par de locais de entrega

In [4]:
# Obtendo somente o endereço de cada pedido
enderecos = df['Endereço']
# Obtendo somente o peso de cada pedido
pesos = df['Peso']

In [5]:
enderecos.head()

0    RUA CORONEL DUARTE DA SILVEIRA, 605
1              RUA BENJAMIN CONSTANT 213
2               RUA PAULINO AFONSO , 477
3                    RUA 24 DE MAIO, 517
4                    RUA ALICE HERVE 356
Name: Endereço, dtype: object

In [6]:
pesos.head()

0    0.0
1    2.0
2    1.2
3    3.0
4    0.5
Name: Peso, dtype: float64

In [7]:
# A API Directions calcula a distância entre dois pontos, e os dados da rota são obtidos através de um web
# service que usa uma solicitação HTTP e retorna um arquivo JSON ou XML, a depender da escolha do usuário

# Criando matriz que armazenará as distâncias
matriz_de_dist = [[0 for x in range(len(pesos))] for y in range(len(pesos))]

# Parte da url que será usada para buscar os dados obtidos pela API
url1 = 'https://maps.googleapis.com/maps/api/directions/json?'
# Chave da API
api_key = 'api_key'

# Loopando por todas as combinações de endereços possíveis
for i in range(len(enderecos)):
    for j in range(len(enderecos)-1, -1, -1):
        # Calculando a distância somente de i até j, e atribuindo esta distância também para j até i
        # Isso diminui o tempo gasto pela API pela metade, além de diminuir o número de solicitações
        if j > i:
            # Obtendo e formatando os endereços dos locais de entrega
            origin = enderecos[i].replace(' ', '+')
            destination = enderecos[j].replace(' ', '+')
            
            # Segunda parte da url que contém a origem, destino, e chave da API 
            url2 = 'origin={}&destination={}&key={}'.format(origin, destination, api_key)
            
            # Juntando as urls para fazer a solicitação
            request = url1 + url2
            
            # Solicitando e obtendo a resposta
            response = urllib.request.urlopen(request)
            
            # A resposta contém diversas informações, portanto é necessário filtrar a distância, 
            # e formatar e converter para números os dados obtidos
            direction = json.load(response)
            distance = direction['routes'][0]['legs'][0]['distance']['text']
            distance = distance.replace(' km', '').replace(' m', '').replace(',', '.')
            distance = float(distance)
            
            # A distância de i até j e de j até i foi estabelecida como a mesma
            matriz_de_dist[i][j] = distance
            matriz_de_dist[j][i] = distance
        elif j == i:
            # Explodindo a distância entre o mesmo pedido, para o solver não pegar esta 'rota'
            matriz_de_dist[i][j] = 1024
            matriz_de_dist[j][i] = 1024
            
# Convertendo a matriz obtida em um DataFrame, para melhor visualização
matriz_de_dist = pd.DataFrame(matriz_de_dist)
matriz_de_dist

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,1024.000,6.700,3.100,2.512,1.300,1.800,852.000,6.800,1.300,3.100,...,6.700,6.800,5.300,462.000,7.000,450.000,2.520,5.100,1.400,5.500
1,6.700,1024.000,4.400,2.509,7.300,9.100,858.000,1.000,7.300,4.400,...,1.000,0.800,6.400,468.000,3.100,456.000,2.517,2.800,6.700,1.500
2,3.100,4.400,1024.000,2.514,4.600,6.400,856.000,3.800,4.600,1.000,...,3.700,3.700,3.700,465.000,4.000,453.000,2.521,2.100,4.100,2.400
3,2.512,2.509,2.514,1024.000,2.521,2.518,3.320,2.513,2.521,2.517,...,2.512,2.512,2.515,2.938,2.516,2.942,16.100,2.514,2.520,2.514
4,1.300,7.300,4.600,2.521,1024.000,3.300,851.000,6.700,1.000,3.000,...,6.600,6.700,6.200,461.000,6.900,449.000,2.522,5.000,0.800,5.400
5,1.800,9.100,6.400,2.518,3.300,1024.000,853.000,8.600,3.100,4.900,...,8.500,8.600,6.000,463.000,8.800,451.000,2.519,7.000,3.200,7.300
6,852.000,858.000,856.000,3.320,851.000,853.000,1024.000,859.000,856.000,858.000,...,859.000,859.000,862.000,393.000,858.000,397.000,3.321,857.000,857.000,858.000
7,6.800,1.000,3.800,2.513,6.700,8.600,859.000,1024.000,6.300,3.300,...,2.600,2.700,5.300,467.000,2.600,455.000,2.523,2.800,5.700,1.900
8,1.300,7.300,4.600,2.521,1.000,3.100,856.000,6.300,1024.000,3.000,...,6.600,6.700,6.200,461.000,6.900,449.000,2.522,5.000,0.800,5.400
9,3.100,4.400,1.000,2.517,3.000,4.900,858.000,3.300,3.000,1024.000,...,3.700,3.700,3.700,465.000,4.000,453.000,2.521,2.100,4.100,2.400


## Criando o modelo de programação linear

In [8]:
matriz_de_dist = matriz_de_dist.to_numpy()

In [15]:
# Declarando o modelo
modelo = pyo.ConcreteModel()

# Adicionando os índices
modelo.I = range(len(matriz_de_dist))
modelo.J = range(len(matriz_de_dist[0]))
modelo.I0 = range(1, len(matriz_de_dist))

# Adicionando os dados do problema no modelo
modelo.m_dist = pyo.Param(modelo.I, modelo.J, initialize=lambda modelo, i, j: matriz_de_dist[i][j])
modelo.pesos = pyo.Param(modelo.I, initialize=lambda modelo, i: pesos[i])

# Adicionando variáveis de decisão
modelo.x = pyo.Var(modelo.I, modelo.J, within=pyo.Binary)
modelo.y = pyo.Var(modelo.I, within=pyo.NonNegativeIntegers)

# Criando função objetivo
def func_obj(mod):
    return pyo.summation(mod.m_dist, mod.x)

modelo.z = pyo.Objective(rule=func_obj, sense=pyo.minimize)

# Adicionando restrições
# Garantir que só saia de cada local de entrega no máximo uma vez
def i_constraint(mod, i):
    return sum(mod.x[i, j] for j in mod.J) <= 1

modelo.restr_i = pyo.Constraint(modelo.I, rule=i_constraint)


# Garantir que só chegue em cada local de entrega no máximo uma vez
def j_constraint(mod, j):
    return sum(mod.x[i, j] for i in mod.I) <= 1

modelo.restr_j = pyo.Constraint(modelo.J, rule=j_constraint)


# Garantir que o caminhão carregue um peso mínimo
def min_weight_constraint(mod):
    return sum(mod.x[i, j] * mod.pesos[j] for j in mod.J for i in mod.I) >= 14

modelo.restr_peso_min = pyo.Constraint(rule=min_weight_constraint)


# Garantir que o caminhão não tenha seu peso máximo excedido
def max_weight_constraint(mod, i):
    return sum(mod.x[i, j] * mod.pesos[j] for j in mod.J for i in mod.I) <= 20

modelo.restr_peso_max = pyo.Constraint(modelo.I, rule=max_weight_constraint)


# Assegurar que sempre volta pra serraria
def serraria_constraint(mod):
    return sum(mod.x[0, i] for i in mod.I) >= 1

modelo.restr_serraria = pyo.Constraint(rule=serraria_constraint)


# Assegurar que sempre que chega em um local, deve sair do local
def conexity_constraint(mod, i):
    return sum(mod.x[i, j] for j in mod.J) - sum(mod.x[j, i] for j in mod.J) >= 0

modelo.restr_conexidade = pyo.Constraint(modelo.I, rule=conexity_constraint)


# Assegurar a conexidade dos locais de entrega
def y_constraint(mod, i, j):
    return mod.y[i] - mod.y[j] + (len(matriz_de_dist) * mod.x[i, j]) <= len(matriz_de_dist) - 1

modelo.restr_y = pyo.Constraint(modelo.I0, modelo.I0, rule=y_constraint)


# Assegurar que o valor da variável y não exceda o número de locais de entrega
def y_constraint2(mod, i):
    return mod.y[i] <= len(matriz_de_dist)

modelo.restr_y2 = pyo.Constraint(modelo.I0, rule=y_constraint2)


# Assegurar que o valor da variável y seja maior que 2
def y_constraint3(mod, i):
    return mod.y[i] >= 2

modelo.restr_y3 = pyo.Constraint(modelo.I0, rule=y_constraint3)

In [16]:
# Usando o solver GLPK para resolver o modelo
resultado = pyo.SolverFactory('glpk').solve(modelo)

In [17]:
# Verificando as rotas escolhidas pelo solver
for i in range(len(matriz_de_dist)):
    for j in range(len(matriz_de_dist[0])):
        if modelo.x[i, j]() == 1.0:
            print(i, j)

0 3
1 21
3 7
7 1
12 23
21 12
23 0


In [18]:
# Mostrando a distância total percorrida
modelo.z()

13.862

## Extraindo as rotas e as exportando

In [19]:
# Função para pegar os locais pelos quais o caminhão deve passar
def get_routes():
    lista = [0]
    origem = 0
    while True:
        for j in range(len(matriz_de_dist[0])):
            if modelo.x[origem, j]() == 1.0:
                lista.append(j)
                origem = j
                if j == 0:
                    return lista
                    

# Colocando os locais de entrega e suas informações em uma lista na ordem em que devem ser percorridos
lista = []
for i in get_routes():
    lista.append(df.iloc[i])
    
# Exportando a lista para um arquivo csv no mesmo diretório
lista = pd.DataFrame(lista)
lista.to_csv('Rotas.csv')